## Motivation

Predicting the dynamics of SARS-Cov-2 infections is essential for quick and effective diagnosis of Covid19,public health planning and mitigating burden on healthcare systems. 
 
## Dataset Description
The John Hopkins University has a dedicated Github repository for Covid19 where it has been publishing time series data for confirmed, recovered and death cases every day for each country.

## Project Overview
Two specific datasets - i.e. time based and country based are created from the JHU data. Exploratoy data analysis is performed on this data followed by modeling using bidirectional convolutional LSTM for forecasting. 

## References

https://blog.clairvoyantsoft.com/covid-19-prediction-using-lstm-cba2fd4fc7fc
https://www.kaggle.com/sandeep2812/covid19-case-study#Importing-Data

https://medium.datadriveninvestor.com/covid19-time-series-forecasting-using-lstm-rnn-753a04944483

https://theblog.github.io/post/convolution-in-autoregressive-neural-networks/

# **1. Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go  
from plotly.subplots import make_subplots
pd.set_option('precision',0)
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Dropout, BatchNormalization,LSTM,SeparableConv1D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import warnings
warnings.filterwarnings('ignore')



# **2. Creating dataset and preprocessing the dataset**

We will create a time series data and country data of Covid19 cases with the data exracted from the JHU repository

### Extracting data from the JHU Github repository

In [2]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df_confirmed = pd.read_csv(url)
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df_deaths = pd.read_csv(url)
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
df_recovered = pd.read_csv(url)


In [3]:
df_confirmed.head(5)

In [4]:
df_deaths.head(5)

In [5]:
df_recovered.head(5)

In [6]:
df_list = [df_confirmed,df_deaths,df_recovered]
cases = ['Confirmed', 'Deaths', 'Recovered', 'Active']
case_color = ['orange','red','green','blue']
case_dict = {cases[i]:case_color[i] for i in range(len(cases))}

### Creating time series data from the extracted data

In [7]:
## creating time series data

time_series_data = pd.DataFrame()
for i in range(len(cases)-1):
    df =  pd.DataFrame(df_list[i][df_list[i].columns[4:]].sum(),columns=[cases[i]])
    time_series_data = pd.concat([time_series_data,df],axis = 1)
time_series_data.index = pd.to_datetime(time_series_data.index,format='%m/%d/%y')
time_series_data['Active'] = time_series_data['Confirmed'] - time_series_data['Deaths'] - time_series_data['Recovered']
time_series_data= time_series_data.rename_axis('ObservationDate').reset_index()

In [8]:
time_series_data.head(10).style.background_gradient(cmap='PuBu')

In [9]:
time_series_data.info()

### Creating country wise data from the extracted data

In [10]:
country_wise_data = pd.DataFrame()
for i in range(len(cases)-1):
    series =  df_list[i][df_list[i].columns[4:]].sum(axis = 1)
    df = pd.concat([df_list[i]['Country/Region'] ,series],axis = 1)
    df = df.groupby('Country/Region').sum().rename(columns = {0:cases[i]})
    country_wise_data = pd.concat([country_wise_data,df],axis = 1)
country_wise_data['Active'] = country_wise_data['Confirmed'] - country_wise_data['Deaths'] - country_wise_data['Recovered']
country_wise_data = country_wise_data.reset_index()

In [11]:
country_wise_data.head(10).style.background_gradient(cmap='PuBu')

In [12]:
country_wise_data.info()

# **3. Exploratory Data Analysis**

In [13]:
country_wise_data = country_wise_data.sort_values(by='Confirmed',ascending=False).reset_index(drop = True)
country_wise_data.head(10).style.background_gradient(cmap='Oranges',subset=["Confirmed"])\
.background_gradient(cmap='Reds',subset=['Deaths'])\
.background_gradient(cmap='Greens',subset=["Recovered"])\
.background_gradient(cmap='Blues',subset=["Active"])

**Comment**: It seems as if the recovery data for US is missing. Based on the above table it can be said that US is the most affected country with respect to number of confirmed cases and fatalities.

In [14]:
#Displaying dates where confirmed case counts were highest
time_series_data = time_series_data.sort_values('ObservationDate', ascending=False).reset_index(drop = True)
time_series_data.head(10).style.background_gradient(cmap='Oranges',subset=["Confirmed"])\
.background_gradient(cmap='Reds',subset=['Deaths'])\
.background_gradient(cmap='Greens',subset=["Recovered"])\
.background_gradient(cmap='Blues',subset=["Active"])

**Comment:** <br>
Some recovery cases are zero so we will fill with previous non-zero values

In [15]:
time_series_data['Recovered'] = time_series_data['Recovered'].replace(to_replace=0, method='bfill')
time_series_data.head(10).style.background_gradient(cmap='Oranges',subset=["Confirmed"])\
.background_gradient(cmap='Reds',subset=['Deaths'])\
.background_gradient(cmap='Greens',subset=["Recovered"])\
.background_gradient(cmap='Blues',subset=["Active"])

In [16]:
count_df = time_series_data.iloc[1,1:]
count_df = pd.DataFrame(count_df).reset_index(level = 0).rename(columns = {'index':'category',1:'count'})
fig = px.bar(count_df, x='count', y='category',
             hover_data=['count'], color='count',
             labels={}, orientation='h',height=400, width = 650)
fig.update_layout(title_text='<b>Confirmed vs Recovered vs Deaths vs Active</b>',title_x=0.5,showlegend = False) 
fig.show()

**Comment:** <br>
The numbers of deaths are pretty low and the recovery numbers are lesser than the confirmed numbers.

In [17]:
df_confirmed = country_wise_data.loc[:,['Country/Region','Confirmed']].sort_values(by = 'Confirmed',ascending = False).reset_index(drop = True).head(10)
df_deaths =    country_wise_data.loc[:,['Country/Region','Deaths']].sort_values(by = 'Deaths',ascending = False).reset_index(drop = True).head(10)
df_active =    country_wise_data.loc[:,['Country/Region','Active']].sort_values(by = 'Active',ascending = False).reset_index(drop = True).head(10)
df_recovered =    country_wise_data.loc[:,['Country/Region','Recovered']].sort_values(by = 'Recovered',ascending = False).reset_index(drop = True).head(10)

In [18]:
fig = px.bar(df_confirmed, x='Confirmed', y='Country/Region',
             hover_data=['Confirmed'], color='Confirmed',
             labels={},orientation='h', height=800, width=650)
fig.update_layout(title_text='<b>Total number of Confirmed cases</b>',title_x=0.5)
fig.show()

In [19]:
fig = px.bar(df_deaths, x='Deaths', y='Country/Region',
             hover_data=['Deaths'], color='Deaths',
             labels={},orientation='h', height=800, width=650)
fig.update_layout(title_text='<b>Total number of Death cases</b>',title_x=0.5)
fig.show()

In [20]:
fig = px.bar(df_active, x='Active', y='Country/Region',
             hover_data=['Active'], color='Active',
             labels={},orientation='h', height=800, width=650)
fig.update_layout(title_text='<b>Total number of Active cases</b>',title_x=0.5)
fig.show()

In [21]:
fig = px.bar(df_recovered, x='Recovered', y='Country/Region',
             hover_data=['Recovered'], color='Recovered',
             labels={},orientation='h', height=800, width=650)
fig.update_layout(title_text='<b>Total number of Recovered cases</b>',title_x=0.5)
fig.show()

In [22]:
# Cases over time
time_series_data = time_series_data.sort_values('ObservationDate').reset_index(drop = True)
time_series_data.iloc[:,1:] = time_series_data.iloc[:,1:].astype('int64')
time_series_data.head(10)

In [23]:
#cases_over_time moving average
time_series_data_avg = time_series_data.copy()
time_series_data_avg.iloc[:,1:] = time_series_data_avg.iloc[:,1:].rolling(window = 7, min_periods = 1).mean()
time_series_data_avg.head(20)

In [24]:
tplot = go.Figure()
for case in cases:
    tplot.add_trace(
        go.Scatter(
            x = time_series_data['ObservationDate'],
            y = time_series_data[case],
            name = case,
            line = dict(color=case_dict[case]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><b>Count</b>: %{y}',
        )
    )
for case in cases:
    tplot.add_trace(
        go.Scatter(
            x = time_series_data_avg['ObservationDate'],
            y = time_series_data_avg[case],
            name = case + " 7-day moving average",
            line = dict(dash = 'dash',color=case_dict[case]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><b>Moving Average Count</b>: %{y}',
            showlegend = False
        )
    )

tplot.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Cases',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, True, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, True]},
                          {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, True, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, True, False, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),type = 'buttons',
             direction="right",
             showactive = True,
             x=-0.25,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
            ]),
             direction="right",
             x=-0.25,
             xanchor="left",
             y=1.39,
             yanchor="top"
        )
    ])

tplot.update_layout(
    height=600, width=1100, 
    title_text="<b>Global cases over time</b>", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="snow",
                            xaxis_title="Observation Date", yaxis_title="Number of Cases")
tplot.show()


In [25]:
# Cases over time
time_series_data_increase = time_series_data.copy()
time_series_data_increase.iloc[:,1:] = time_series_data_increase.iloc[:,1:].diff(1)
time_series_data_increase_avg  = time_series_data_increase.copy()
time_series_data_increase_avg.iloc[:,1:] = time_series_data_increase_avg.iloc[:,1:].rolling(window = 7, min_periods = 1).mean()

In [26]:
fig = make_subplots(rows=len(cases), cols=1, vertical_spacing=0.2, horizontal_spacing=0.04, # shared_yaxes=True,
                           subplot_titles=('<b>Confirmed</b>','<b>Active</b>','<b>Recovered</b>','<b>Deaths</b>'),
                            x_title="Observation Date", y_title="Number of Cases")


for i in range(len(cases)):

    fig.add_trace( go.Bar(
                x = time_series_data_increase['ObservationDate'],
                y = time_series_data_increase[cases[i]],
                name = cases[i],
                hovertemplate ='<br><b>Date</b>: %{x}'+'<br><b>Count</b>: %{y}',
                marker = dict(color = case_dict[cases[i]])
            ),row = i+1, col = 1)

    fig.add_trace( go.Scatter(
                x = time_series_data_increase_avg['ObservationDate'],
                y = time_series_data_increase_avg[cases[i]],
                name = cases[i],
                hovertemplate ='<br><b>Date</b>: %{x}'+'<br><b>7-day average</b>: %{y}',
                showlegend=False,
                line=dict(dash="dash", color=case_color[i])
            ),row = 1+i, col = 1)
fig.update_layout(
    height=800, width=1100, 
    title_text="<b>Daily increase in global cases over time</b>", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="snow")
fig.show()



In [27]:
fig,ax = plt.subplots(1,4,figsize = (20,5))
for i in range(len(cases)):
    sns.set_style('whitegrid')
    sns.distplot(time_series_data[cases[i]], kde = True, rug = True, bins = 25,ax =ax[i],color = case_color[i])
    ax[i].set_xlabel( cases[i]+ ' cases')
    ax[i].set_ylabel('Density')
    ax[i].set_title('Distribution plot for ' + cases[i]+ ' cases')
fig.tight_layout()
plt.suptitle('Distribution of Cases',fontsize = 15,y = 1.1)
plt.show()


**Comment**:<br>
The distribution plots are skewed. So appropriate normalization technqiues need to be applied before modelling.

In [28]:

# m = np.mean(time_series_data.iloc[:,1:], axis=0) # array([16.25, 26.25])
# std = np.std(time_series_data.iloc[:,1:], axis=0) # array([17.45530005, 22.18529919])
# md = np.median(time_series_data.iloc[:,1:],axis = 0)
# p75 = np.percentile(time_series_data.iloc[:,1:],75,axis = 0)
# p25 = np.percentile(time_series_data.iloc[:,1:],25,axis = 0)

In [29]:
# time_series_normalized = time_series_data.copy()
# time_series_normalized.iloc[:,1:] = np.tanh((time_series_normalized.iloc[:,1:] - md) / (p75 - p25)) #normalization and feature scaling for train set

In [30]:
# fig,ax = plt.subplots(1,4,figsize = (20,5))
# for i in range(len(cases)):
#     sns.set_style('whitegrid')
#     sns.distplot(time_series_normalized[cases[i]], kde = True, rug = True, bins = 25,ax =ax[i],color = case_color[i])
#     ax[i].set_xlabel( cases[i]+ ' cases')
#     ax[i].set_ylabel('Density')
#     ax[i].set_title('Distribution plot for ' + cases[i]+ ' cases')
# fig.tight_layout()
# plt.suptitle('Distribution of Normalized time series data',fontsize = 15,y = 1.1)
# plt.show()

# **4.Predictive modelling using CNN + Bi-Directional LSTM**


### Creating dataset for predictive modelling

In [31]:
dataset = time_series_data.iloc[:,1].values #using only confirmed cases
dataset.shape

In [32]:
#Feature scaling
split = round(0.8*len(dataset))
dataset = dataset.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(dataset[:split]) #fit only on the training data which is the first 80%
dataset_n = scaler.transform(dataset).flatten()
dataset_n.shape


In [33]:
def create_dataset(df,previous,split_ratio):
    X, Y = [], []
    for i in range(len(df)-previous):
        a = df[i:(i+previous)]
        X.append(a)
        y = df[i+previous]
        Y.append(y)
    X = np.array(X).reshape(-1,T,1)
    Y = np.array(Y)
    N = len(X)
    split = round(split_ratio*len(df)) 
    X_train = X[:split]
    X_test = X[split:]
    Y_train = Y[:split]
    Y_test = Y[split:]
    print("X.shape", X.shape, "Y.shape", Y.shape)
    print("X_train.shape", X_train.shape, "Y_train.shape", Y_train.shape)
    print("X_test.shape", X_test.shape, "Y_test.shape", Y_test.shape)
    return X,X_train,X_test,Y,Y_train,Y_test

In [34]:
T = 5  #number of past days used to predict the value for the current day
X,X_train,X_test,Y,Y_train,Y_test = create_dataset(dataset_n,T,0.8) #80% of data for training and 20% for testing

### Building the model architecture

In [35]:
def plotLearningCurve(history,epochs,text):
  epochRange = range(1,epochs+1)
  plt.figure(figsize = (5,5))
  plt.plot(epochRange,history.history['loss'])
  plt.plot(epochRange,history.history['val_loss'])
  plt.title('Model Loss for ' + text)
  plt.xlabel('Epoch', fontsize = 20)
  plt.ylabel('Loss', fontsize = 20)
  plt.legend(['Training set','Validation set'])
  plt.show()

In [36]:
def lstm_model(previous):
    i = Input(shape=(previous,1)) #input shape is n-timesteps x n-features
    x = Conv1D(filters=64, kernel_size= 5, strides=3, padding="causal", activation="relu")(i)
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.4)(x)
    x = LSTM(64,return_sequences=True)(x)
    x = Dropout(0.4)(x)
    x = LSTM(64)(x)
    x = Dropout(0.2)(x)
    x = Dense(1)(x)
    model = Model(i, x)
    model.summary()
    return model

In [37]:
model = lstm_model(T)

In [38]:
 model.compile(loss = 'mse',
              optimizer = 'adam')

In [39]:
batchsize = 64
epochs = 100
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.8, 
                                            min_lr=1e-10)
early_stop = EarlyStopping(monitor='val_loss',patience=50,restore_best_weights=True)
r = model.fit(X_train,
                    Y_train,
                    batch_size=batchsize,
                    epochs=epochs,
                    validation_split=0.2,
                    shuffle=False, #time_series
                    callbacks=[learning_rate_reduction])

In [40]:
n_epochs = len(r.history['loss'])

In [41]:
print("Train score:", model.evaluate(X_train,Y_train))
print("Test score:", model.evaluate(X_test,Y_test))


In [42]:
plotLearningCurve(r,n_epochs,text = 'CNN+LSTM model')

### Predicting test data

In [43]:
Y_pred = model.predict(X_test)
Y_pred = scaler.inverse_transform(Y_pred)
Y_test = scaler.inverse_transform(Y_test.reshape(-1,1))
Y_train = scaler.inverse_transform(Y_train.reshape(-1,1))
plt.plot(Y_pred, color='red')
plt.plot(Y_test, color='blue')
plt.title('Actual vs. Predicted Covid Cases (Test Data)')
plt.ylabel('Number of Cases')
plt.xlabel('Day')
plt.legend(['predicted', 'actual'])

### Forecasting future values

In [44]:
Y_train = Y_train.reshape(-1)
Y_test = Y_test.reshape(-1)
Y_pred = Y_pred.reshape(-1)

In [45]:
date_array = np.array(time_series_data['ObservationDate'].values)
date_train = date_array[:split]
date_test = date_array[split:]

In [46]:

trace1 = go.Scatter(
    x = date_train,
    y = Y_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = Y_pred,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = Y_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "<b>Confirmed Cases</b>",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Cases"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

### Forecasting

In [47]:
dataset_n[-T:]

In [48]:
T = 5
prediction_list = dataset_n[-T:]  #take last 100 values from the dataset
num_prediction = 30 #predicting cases for next 1 month
for _ in range(num_prediction):
    x = prediction_list[-T:]
    x = x.reshape((1,T, 1))
    out = model.predict(x)[0][0]
    prediction_list = np.append(prediction_list, out) #appending the prediction
prediction_list = prediction_list[T-1:] 
prediction_list = scaler.inverse_transform(np.array(prediction_list).reshape(-1,1)).reshape(-1)


last_date = time_series_data['ObservationDate'].values[-1]
prediction_dates = np.array(pd.date_range(last_date, periods=num_prediction+1))



In [49]:
trace1 = go.Scatter(
    x = date_array,
    y = dataset.reshape(-1),
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = prediction_dates,
    y = prediction_list,
    mode = 'lines',
    name = 'Forecast'
)

layout = go.Layout(
    title = "<b>Confirmed Cases</b>",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Cases"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()